# 사용할 라이브러리

In [2]:
import torch
print(torch.cuda.is_available())  # True여야 정상
print(torch.version.cuda)  # CUDA 버전 확인
print(torch.backends.cudnn.version())  # cuDNN 버전 확인


True
12.4
90100


In [3]:
# 사용할 라이브러리

import os
import glob
import json
import torch
import torchaudio
import numpy as np
from torch.utils.data import Dataset, DataLoader

from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer
)

import evaluate

print("라이브러리 임포트 완료")


/home/kookmin/venvs/seungminENV_pip/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 임포트 완료


In [4]:
# torchaudio - backend

import torchaudio
torchaudio.set_audio_backend("sox_io")
print("torchaudio backend set to sox_io")



torchaudio backend set to sox_io


/tmp/ipykernel_1326908/2652432896.py:4: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")


In [5]:
import torchaudio

print("현재 backend:", torchaudio.get_audio_backend())  # 제거 가능



현재 backend: None


/tmp/ipykernel_1326908/2392935519.py:3: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  print("현재 backend:", torchaudio.get_audio_backend())  # 제거 가능


In [6]:
import torch, torchaudio
print("torch version:", torch.__version__)
print("torchaudio version:", torchaudio.__version__)


torch version: 2.6.0+cu124
torchaudio version: 2.6.0+cu124


# hyper parameter & 저장된 데이터 경로 & 저장 경로


In [21]:
# hyper parameter & 저장된 데이터 경로 & 저장 경로


# 데이터 경로
DATA_DIR_TRAIN = '/data/seungmin/training'
DATA_DIR_VAL = '/data/seungmin/validation'

# 저장 경로
MODEL_SAVE_DIR = '/data/seungmin/model'

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 하이퍼파라미터 설정 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 
BATCH_SIZE = 1 # 8로 했더니 cuda out of mem 떴음
GRAD_ACCUMULATION_STEPS = 16 # 2로 했더니 cuda out of mem떴음
LEARNING_RATE = 1e-5
NUM_EPOCHS = 1
WARMUP_RATIO = 0.1
MAX_DURATION_SEC = 30  # 최대 입력 길이 30초
SAMPLE_RATE = 16000

print("설정 완료")


설정 완료


# 동작 가능한지 테스트해보려고 실행한 것임 이 셀은 실행 ㄴㄴㄴㄴ
## 데이터셋 중 일부만 사용

In [8]:
# DEBUG_MODE = True
# DEBUG_TRAIN_LIMIT = 100   # 훈련 샘플 제한
# DEBUG_VAL_LIMIT = 20      # 검증 샘플 제한

# # 데이터셋 생성 (파일 매칭 및 필터링)
# def build_file_list(data_dir, limit=None):
#     """
#     주어진 data_dir 내에서 source의 wav 파일과 labels의 json 파일을 매칭하여 리스트로 반환.
#     - 각 wav 파일을 torchaudio.load()로 로딩 테스트할 때, format="wav"를 명시하여 시도합니다.
#     - 문제가 발생하면 fallback으로 soundfile을 사용합니다.
#     - 두 방법 모두 실패하면 해당 파일은 건너뜁니다.
#     - limit가 지정되면 해당 개수만 반환합니다.
#     """
#     file_list = []
#     audio_pattern = os.path.join(data_dir, 'source', 'eng', '*', '*', '*.wav')
#     audio_files = glob.glob(audio_pattern)
    
#     for audio_path in audio_files:
#         json_path = audio_path.replace(os.path.join('source', 'eng'), os.path.join('labels', 'eng')).replace('.wav', '.json')
#         if not os.path.exists(json_path):
#             print(f"JSON 파일이 존재하지 않습니다: {json_path}")
#             continue
        
#         loaded = False
#         try:
#             # format 인자를 추가해 wav 파일임을 명시합니다.
#             _waveform, sr = torchaudio.load(audio_path, format="wav")
#             loaded = True
#         except Exception as e:
#             print(f"torchaudio.load() 실패: {audio_path} 에서 오류 발생: {e}")
        
#         if not loaded:
#             try:
#                 import soundfile as sf
#                 _waveform, sr = sf.read(audio_path)
#                 _waveform = torch.tensor(_waveform).float()
#                 loaded = True
#                 print(f"soundfile 로드 성공: {audio_path}")
#             except Exception as e:
#                 print(f"soundfile 로드 실패: {audio_path} 에서 오류 발생: {e}")
#                 continue
        
#         try:
#             with open(json_path, 'r', encoding='utf-8') as f:
#                 data = json.load(f)
#             text = data["06_transcription"]["1_text"]
#         except Exception as e:
#             print(f"JSON 파일 로드 중 오류 발생: {json_path}, {e}")
#             continue
        
#         file_list.append({
#             "audio_path": audio_path,
#             "json_path": json_path,
#             "text": text
#         })
        
#         if limit is not None and len(file_list) >= limit:
#             break
#     return file_list


# # 훈련 및 검증 데이터 파일 리스트 생성 (DEBUG_MODE에 따라 일부만 사용)
# train_files = build_file_list(DATA_DIR_TRAIN, limit=DEBUG_TRAIN_LIMIT if DEBUG_MODE else None)
# val_files = build_file_list(DATA_DIR_VAL, limit=DEBUG_VAL_LIMIT if DEBUG_MODE else None)

# print("예시 훈련 파일:", train_files[0])
# print("예시 검증 파일:", val_files[0])
# print(f"훈련 파일 수: {len(train_files)}")
# print(f"검증 파일 수: {len(val_files)}")
# print("DEBUG_MODE가 활성화되어 일부 샘플만 사용합니다.")


# 데이터 파일 리스트 생성 (wav와 JSON 매칭)

In [ ]:
# 데이터 파일 리스트 생성 (wav와 JSON 매칭)

def build_file_list(data_dir):
    """
    주어진 data_dir 내에서 source의 wav 파일과 labels의 json 파일을 매칭하여 리스트로 반환
    """
    file_list = []
    
    # wav 파일 경로: data/seungmin/{training, validation}/source/eng/{도메인}/{강의ID}/*.wav
    audio_pattern = os.path.join(data_dir, 'source', 'eng', '*', '*', '*.wav')
    audio_files = glob.glob(audio_pattern)
    
    for audio_path in audio_files:
        # source에서 labels 경로로 변환 (source -> labels, .wav -> .json)
        json_path = audio_path.replace(os.path.join('source', 'eng'), os.path.join('labels', 'eng')).replace('.wav', '.json') # source/eng → labels/eng 로 변환하고 & .wav → .json 으로 변경하여 WAV 파일과 매칭되는 JSON 파일 경로 생성
        if os.path.exists(json_path):
            try:
                with open(json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                # JSON 내 "06_transcription"에서 실제 전사 텍스트 추출
                text = data["06_transcription"]["1_text"]
                file_list.append({
                    "audio_path": audio_path,
                    "json_path": json_path,
                    "text": text
                })
            except Exception as e:
                print(f"파일 {json_path} 로드 중 오류 발생: {e}")
        else:
            print(f"JSON 파일이 존재하지 않음 : {json_path}")
    return file_list

# 훈련 및 검증 데이터 파일 리스트 생성
train_files = build_file_list(DATA_DIR_TRAIN)
val_files = build_file_list(DATA_DIR_VAL)

print("예시 훈련 파일:", train_files[0])

print(f"훈련 파일 수: {len(train_files)}")
print(f"검증 파일 수: {len(val_files)}")


예시 훈련 파일: {'audio_path': '/data/seungmin/training/source/eng/comp/C03102/U00414.wav', 'json_path': '/data/seungmin/training/labels/eng/comp/C03102/U00414.json', 'text': '낮고 그다음에 프롭의 추진력이 좀 세야 되 강해야 되기 때문에 코드가 긴 (sf)/(에스에프) 형상 이런 형상을 기준으로 선 선택하는게 좀 더 좋고 저중량 같은 경우에는 코드가 좀 짧은 이런 형상을 선택하는 것이 조금 더 좋을 수 있다 볼 수 있는 거예요.'}
훈련 파일 수: 357212
검증 파일 수: 40387


# 오디오 로딩 및 전처리(custom dataset class 정의)


In [ ]:
class SpeechDataset(Dataset):
    def __init__(self, file_list, processor, sample_rate=SAMPLE_RATE, max_duration=MAX_DURATION_SEC):
        """
        file_list: build_file_list()로 생성한 원본 파일 리스트
        processor: Wav2Vec2Processor 객체
        sample_rate: 목표 샘플링 레이트
        max_duration: 최대 길이(초)
        
        생성자에서 각 파일을 미리 로딩 테스트하여 유효한 파일만 포함.
        """
        valid_list = []
        for item in file_list:
            audio_path = item['audio_path']
            loaded = False
            try:
                _waveform, sr = torchaudio.load(audio_path, format="wav")
                loaded = True
            except Exception as e:
                try:
                    import soundfile as sf
                    waveform, sr = sf.read(audio_path)
                    _ = torch.tensor(waveform).float()
                    loaded = True
                    print(f"파일 {audio_path}는 soundfile로 로드 가능합니다.")
                except Exception as e2:
                    print(f"파일 {audio_path} 필터링됨: torchaudio 실패({e}), soundfile 실패({e2})")
                    continue
            valid_list.append(item)
        self.file_list = valid_list
        self.processor = processor
        self.sample_rate = sample_rate
        self.max_length = sample_rate * max_duration
        print(f"SpeechDataset 생성 완료: {len(self.file_list)}개의 유효한 샘플 포함.")

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        item = self.file_list[idx]
        audio_path = item['audio_path']
        loaded = False
        try:
            speech_tensor, sr = torchaudio.load(audio_path, format="wav")
            loaded = True
        except Exception as e:
            print(f"__getitem__: torchaudio.load() 실패: {audio_path} ({e})")
        if not loaded:
            try:
                import soundfile as sf
                waveform, sr = sf.read(audio_path)
                speech_tensor = torch.tensor(waveform).float()
                if speech_tensor.dim() == 1:
                    speech_tensor = speech_tensor.unsqueeze(0)
                loaded = True
                print(f"__getitem__: soundfile 로드 성공: {audio_path}")
            except Exception as e:
                print(f"__getitem__: 파일 로드 실패: {audio_path} ({e})")
                dummy_audio = np.zeros(self.max_length, dtype=np.float32)
                return {"input_values": dummy_audio, "labels": ""}
        if sr != self.sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)
            speech_tensor = resampler(speech_tensor)
        speech_array = speech_tensor.squeeze().numpy()
        if len(speech_array) > self.max_length:
            speech_array = speech_array[:self.max_length]
        text = item['text']
        # 키 이름을 변경하여 모델의 기대입력과 맞춤
        return {"input_values": speech_array, "labels": text}




print("SpeechDataset class 정의 완료")


SpeechDataset class 정의 완료


# model, processor 로딩

In [11]:
# model, processor 로딩

processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

print("모델 및 프로세서 로드 완료")


모델 및 프로세서 로드 완료


# 데이터셋 인스턴스 생성

In [12]:
# 데이터셋 인스턴스 생성

train_dataset = SpeechDataset(train_files, processor, sample_rate=SAMPLE_RATE, max_duration=MAX_DURATION_SEC)
val_dataset = SpeechDataset(val_files, processor, sample_rate=SAMPLE_RATE, max_duration=MAX_DURATION_SEC)

print(train_dataset[0])
print(f"훈련 데이터셋 샘플 수: {len(train_dataset)}")
print(f"검증 데이터셋 샘플 수: {len(val_dataset)}")


SpeechDataset 생성 완료: 357212개의 유효한 샘플 포함.
SpeechDataset 생성 완료: 40387개의 유효한 샘플 포함.
{'input_values': array([-0.00125122,  0.00015259, -0.00076294, ...,  0.00091553,
       -0.00012207,  0.00088501], shape=(282633,), dtype=float32), 'labels': '낮고 그다음에 프롭의 추진력이 좀 세야 되 강해야 되기 때문에 코드가 긴 (sf)/(에스에프) 형상 이런 형상을 기준으로 선 선택하는게 좀 더 좋고 저중량 같은 경우에는 코드가 좀 짧은 이런 형상을 선택하는 것이 조금 더 좋을 수 있다 볼 수 있는 거예요.'}
훈련 데이터셋 샘플 수: 357212
검증 데이터셋 샘플 수: 40387


In [13]:
# only 디버깅용


# 빈 샘플이 있는지 확인
empty_samples = [i for i, sample in enumerate(train_dataset) if len(sample["input_values"]) == 0 or sample["labels"] == ""]
print(f"비어 있는 샘플 개수: {len(empty_samples)}")


비어 있는 샘플 개수: 0


# 데이터 collator 정의 (dynamic padding 포함)

In [14]:
# # 데이터 collator 정의 (dynamic padding 포함)

# # '비어 있는 샘플 개수: 0'라고 나오는데, 자꾸 파인튜닝 코드 실행 시 '배치의 샘플이 유효하지 않다'고 출력되어서 디버깅 코드 추가하였음

# def data_collator(batch):
#     # 배치가 딕셔너리 형태이면 이미 collate된 상태임
#     if isinstance(batch, dict):
#         speeches = batch.get("speech", [])
#         texts = batch.get("text", [])
#     else:
#         # 리스트인 경우 개별 샘플을 필터링
#         valid_batch = [
#             item for item in batch 
#             if item and "speech" in item and "text" in item and len(item["speech"]) > 0 and item["text"]
#         ]
#         if len(valid_batch) == 0:
#             raise ValueError("모든 배치의 샘플이 유효하지 않습니다.")
#         speeches = [item["speech"] for item in valid_batch]
#         texts = [item["text"] for item in valid_batch]

#     print(f"📌 최종 배치 크기: {len(speeches)}")
#     print(f" 첫 번째 wav 길이: {len(speeches[0]) if speeches else 'N/A'}")
#     print(f" 첫 번째 텍스트: {texts[0] if texts else 'N/A'}")
    
#     inputs = processor(speeches, text=texts, sampling_rate=SAMPLE_RATE, padding=True, return_tensors="pt")
#     return inputs



# print("data_collator 정의 완료")


In [15]:
def data_collator(features):
    # features는 {"input_values": ..., "labels": ...} 형태의 딕셔너리 리스트입니다.
    if isinstance(features, list):
        if features and isinstance(features[0], dict):
            # "input_values" 키가 있는지 확인합니다.
            if "input_values" in features[0]:
                input_values = [f["input_values"] for f in features]
                labels = [f["labels"] for f in features]
            else:
                raise ValueError("키 'input_values'가 존재하지 않습니다.")
        else:
            raise ValueError("리스트 내 요소가 dict 형태가 아닙니다.")
    elif isinstance(features, dict):
        if "input_values" in features:
            input_values = features.get("input_values", [])
            labels = features.get("labels", [])
        else:
            raise ValueError("dict 형태지만 'input_values' 키가 없습니다.")
    else:
        raise ValueError("배치의 형식이 올바르지 않습니다.")
    
    # processor를 사용해 동적 패딩 수행
    batch = processor(input_values, text=labels, sampling_rate=SAMPLE_RATE, padding=True, return_tensors="pt")
    return batch


In [16]:
# 모델, 프로세서 로딩 이후에 gradient checkpointing 활성화
model.gradient_checkpointing_enable()

# Training aruments 설정

In [22]:
# Training arguments 설정정

# 하이퍼파라미터 설정은 위에서 하기 
training_args = TrainingArguments(
    output_dir=MODEL_SAVE_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    
    eval_strategy="epoch", # evaluation_strategy는 구버젼에서만 잘됨
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    fp16=True, # mixed precision
    logging_steps=100,
    save_total_limit=2,  # best 결과 + latest 결과만 저장
    load_best_model_at_end=True,
    dataloader_num_workers=0
)



print("TrainingArguments 설정 완료")


TrainingArguments 설정 완료


# 평가 지표 함수 정의 (WER 기반)
evaluation metrics

In [18]:
# 평가 지표 함수 정의 (WER 기반)

wer_metric = evaluate.load("wer") # evaluate 라이브러리를 사용해 WER metric 로드

def compute_metrics(pred):
    """
        예측 결과와 라벨을 processor를 사용하여 디코딩 후 WER 계산
    """
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = processor.batch_decode(pred_ids)
    
    # -100은 무시 처리: label_ids 내의 값들을 디코딩 
    label_ids = pred.label_ids
    label_str = processor.batch_decode(label_ids, group_tokens=False) # group_tokens=False 옵션이 뭔지 잘 모름
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str) # prediction vs label
    return {"wer": wer}

print("evaluation metric 함수 정의 완료")


evaluation metric 함수 정의 완료


In [19]:
# # 데이터셋 확인
# for i in range(len(train_dataset)):
#     sample = train_dataset[i]
#     if sample is None:
#         print(f"인덱스 {i}는 유효하지 않습니다.")


# empty_samples = [i for i, sample in enumerate(train_dataset) if not sample["speech"] or not sample["text"]]
# print(f"비어 있는 샘플 개수: {len(empty_samples)}")

sample = train_dataset[0]
print(f"첫 번째 샘플 speech 길이: {len(sample['input_values'])}")
print(f"첫 번째 샘플 text: {sample['labels']}")


첫 번째 샘플 speech 길이: 282633
첫 번째 샘플 text: 낮고 그다음에 프롭의 추진력이 좀 세야 되 강해야 되기 때문에 코드가 긴 (sf)/(에스에프) 형상 이런 형상을 기준으로 선 선택하는게 좀 더 좋고 저중량 같은 경우에는 코드가 좀 짧은 이런 형상을 선택하는 것이 조금 더 좋을 수 있다 볼 수 있는 거예요.


# Trainer 객체 생성  & 학습 실행

In [ ]:
# Trainer 객체 생성  & 학습 실행

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
    # tokenizer=processor
    # dataloader_pin_memory=False # 최신버젼에서는 미지원이라
    # remove_unused_columns=False
)

print("Trainer 객체 생성 완료")
print("파인튜닝 시작")


train_result = trainer.train()
print("파인튜닝 완료")



Trainer 객체 생성 완료
파인튜닝 시작


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

: 

# 모델 저장

In [ ]:
# 모델 저장

# 최종 모델과 프로세서 저장 (.pt 형식은 아니지만 Hugging Face 포맷으로 저장 후 변환 가능)
model.save_pretrained(MODEL_SAVE_DIR)
processor.save_pretrained(MODEL_SAVE_DIR)

print(f"모델이 {MODEL_SAVE_DIR}에 저장됨")
